In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.style as style
import os
import geopandas as gpd
import geoplot
import mapclassify
from tensorflow.keras.utils import get_file
from scipy import optimize
style.use('default')

In [2]:
df = pd.read_excel('../data/Microdados-2020.07.03.xlsx')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529284 entries, 0 to 529283
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   regiao                  529284 non-null  object        
 1   estado                  529155 non-null  object        
 2   municipio               523614 non-null  object        
 3   coduf                   529284 non-null  int64         
 4   codmun                  525672 non-null  float64       
 5   codRegiaoSaude          523614 non-null  float64       
 6   nomeRegiaoSaude         523614 non-null  object        
 7   data                    529284 non-null  datetime64[ns]
 8   semanaEpi               529284 non-null  int64         
 9   populacaoTCU2019        527226 non-null  object        
 10  casosAcumulado          529284 non-null  int64         
 11  casosNovos              529284 non-null  int64         
 12  obitosAcumulado         529284

In [4]:
df['data']= pd.to_datetime( df['DATA AJUSTADA'], format='%d/%m/%Y')

KeyError: 'DATA AJUSTADA'

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529284 entries, 0 to 529283
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   regiao                  529284 non-null  object        
 1   estado                  529155 non-null  object        
 2   municipio               523614 non-null  object        
 3   coduf                   529284 non-null  int64         
 4   codmun                  525672 non-null  float64       
 5   codRegiaoSaude          523614 non-null  float64       
 6   nomeRegiaoSaude         523614 non-null  object        
 7   data                    529284 non-null  datetime64[ns]
 8   semanaEpi               529284 non-null  int64         
 9   populacaoTCU2019        527226 non-null  object        
 10  casosAcumulado          529284 non-null  int64         
 11  casosNovos              529284 non-null  int64         
 12  obitosAcumulado         529284

In [8]:
fltr = df['MUNICÍPIO RESIDÊNCIA AJUSTADO'].str.lower() == 'DOURADOS'.lower()
df_dou = df.loc[fltr, :]

KeyError: 'MUNICÍPIO RESIDÊNCIA AJUSTADO'

In [ ]:
df_dou

,SEXO,MUNICÍPIO RESIDÊNCIA AJUSTADO,MUNICÍPIO NOTIFICAÇÃO AJUSTADO,DATA AJUSTADA,FAIXA_ETARIA,IDADE,TIPO DE TESTE,STATUS,DETALHAMENTO STATUS,Febre,...,Diabetes,Imunosupressão,Cardio,Gestante alto risco,RESULTADO EXAME,SINTOMAS_2,COMORBIDADES_2,É PROFISSIONAL DE SAÚDE?,É PROFISSIONAL DE FORÇA DE SEGURANÇA?,data
1,FEMININO,DOURADOS,DOURADOS,11/06/2020,40 a 49 Anos,49,RT-PCR,CONFIRMADO,RECUPERADO,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Sim,Não,2020-06-11
4,MASCULINO,DOURADOS,DOURADOS,28/06/2020,40 a 49 Anos,43,RT-PCR,CONFIRMADO,EM TRATAMENTO DOMICILIAR,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Não,Não,2020-06-28
5,MASCULINO,DOURADOS,DOURADOS,22/05/2020,10 a 19 Anos,19,RT-PCR,CONFIRMADO,RECUPERADO,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Não,Não,2020-05-22
13,MASCULINO,DOURADOS,DOURADOS,05/06/2020,20 a 29 Anos,28,RT-PCR,CONFIRMADO,RECUPERADO,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Não,Não,2020-06-05
21,MASCULINO,DOURADOS,DOURADOS,23/06/2020,40 a 49 Anos,47,TESTE RÁPIDO,CONFIRMADO,EM TRATAMENTO DOMICILIAR,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Não,Não,2020-06-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9374,MASCULINO,DOURADOS,DOURADOS,04/06/2020,30 a 39 Anos,31,RT-PCR,CONFIRMADO,RECUPERADO,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Não,Não,2020-06-04
9377,MASCULINO,DOURADOS,DOURADOS,05/06/2020,40 a 49 Anos,47,TESTE RÁPIDO,CONFIRMADO,RECUPERADO,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Sim,NaN,2020-06-05
9380,FEMININO,DOURADOS,DOURADOS,18/06/2020,20 a 29 Anos,26,RT-PCR,CONFIRMADO,EM TRATAMENTO DOMICILIAR,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Não,Não,2020-06-18
9381,FEMININO,DOURADOS,DOURADOS,28/06/2020,30 a 39 Anos,37,TESTE RÁPIDO,CONFIRMADO,EM TRATAMENTO DOMICILIAR,NaN,...,NaN,NaN,NaN,NaN,POSITIVO,-,-,Não,Não,2020-06-28


In [ ]:
df_dou_dia = df_dou.groupby('data')

In [ ]:
df_dou['RESULTADO EXAME'].value_counts()

POSITIVO    2869
Name: RESULTADO EXAME, dtype: int64

In [ ]:
df_dou.shape

(2869, 26)

In [ ]:
df['RESULTADO EXAME'].value_counts()

POSITIVO    9379
Name: RESULTADO EXAME, dtype: int64